In [19]:
import cv2
import os
import json
import numpy as np

images_dir = './samples'
input = 'input.json'
output = 'output.json'
with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 4
# Load the image
original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))

# Resize the image
original_image = cv2.resize(original_image, (0, 0), fx=0.15, fy=0.15)
image = original_image.copy()

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply median blur
blur = cv2.GaussianBlur(gray, (5, 5), 0)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
blur = clahe.apply(blur)

# Harris Corner Detection
harris_image = image.copy()
harris_num_legos = 0
# list of contours
dst = cv2.cornerHarris(blur, 2, 3, 0.04)
dst = cv2.dilate(dst, None)
threshold = 0.01 * dst.max()
corner_image = image.copy()
corner_image[dst > threshold] = [0,0,255]
ret, dst_thresh = cv2.threshold(dst, threshold, 255, 0)
dst_thresh = np.uint8(dst_thresh)
contours, _ = cv2.findContours(dst_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 50:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(harris_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        harris_num_legos += 1


canny_image = image.copy()
cannys_found_contours = [] 

edges = cv2.Canny(blur, 30, 50)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    area = cv2.contourArea(contour)
    x, y, w, h = cv2.boundingRect(contour)
    
    if area > 10:
        to_place = True
        cannys_found_contours_copy = cannys_found_contours.copy()
        """  for existing_contour_info in cannys_found_contours:
            existing_contour = existing_contour_info['contour']
            existing_x, existing_y, existing_w, existing_h = existing_contour_info['x'], existing_contour_info['y'], existing_contour_info['w'], existing_contour_info['h']
            
            intersect_x1 = max(x, existing_x)
            intersect_y1 = max(y, existing_y)
            intersect_x2 = min(x + w, existing_x + existing_w)
            intersect_y2 = min(y + h, existing_y + existing_h)
            intersect_area = max(0, intersect_x2 - intersect_x1) * max(0, intersect_y2 - intersect_y1)



            if intersect_area > 0:
                if((w*h) > (existing_w * existing_h)):
                    cannys_found_contours_copy.remove(existing_contour_info)    
                else:
                    to_place = False
                    break
                 """

        if to_place:
            contour_info = {
                'x': x,
                'y': y,
                'w': w,
                'h': h,
                'contour': contour 
            }

            cannys_found_contours_copy.append(contour_info)
            cannys_found_contours = cannys_found_contours_copy

for found_contour in cannys_found_contours:
    x, y, w, h = found_contour['x'], found_contour['y'], found_contour['w'], found_contour['h']
    cv2.rectangle(canny_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    


cv2.imshow('Original Image', original_image)
cv2.imshow('Corner Detection', corner_image) 
cv2.imshow('Blur', blur)
cv2.imshow('Harris Corner Detection', harris_image)
cv2.imshow('Canny Edge Detection', canny_image)
cv2.imshow('Canny Corner Detection', edges)


cv2.waitKey(0)
cv2.destroyAllWindows()


In [102]:
import cv2
import os
import json
import numpy as np

images_dir = './samples'
input = 'input.json'
output = 'output.json'
with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 22
# Load the image
original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))

# Resize the image
original_image = cv2.resize(original_image, (0, 0), fx=0.15, fy=0.15)
image = original_image.copy()

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# Flatten the HSV image to a 1D array
hsv_flattened = hsv.reshape((-1, 3))

# Mask out non-zero pixels (non-black pixels)
hsv_flattened = hsv_flattened[hsv_flattened[:, 2] > 0]

# Calculate the median color (background) in the image
background_color = np.median(hsv_flattened, axis=0)
print(background_color)
tolerance = 110
lower_limit_0 = max(background_color[0] - tolerance,0)
upper_limit_0 = background_color[0] + tolerance
lower_limit_1 = max(background_color[1] - tolerance,0)
upper_limit_1 = background_color[1] + tolerance
lower_limit_2 = max(background_color[2] - tolerance,0)
upper_limit_2 = background_color[2] + tolerance

lower_background = np.array([lower_limit_0, lower_limit_1, lower_limit_2])
upper_background = np.array([upper_limit_0, upper_limit_1, upper_limit_2])


blur = cv2.GaussianBlur(hsv, (5, 5), 0)
grey_mask = cv2.inRange(blur, lower_background, upper_background)

non_grey_mask = cv2.bitwise_not(grey_mask)

result = cv2.bitwise_and(image, image, mask=non_grey_mask)

contours, _ = cv2.findContours(non_grey_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if(w*h < 400):
        continue
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the result
cv2.imshow('Mask OG', grey_mask)
cv2.imshow('Mask', non_grey_mask)
cv2.imshow('hsv', hsv)
cv2.imshow('blur', blur)
cv2.imshow('Segmented LEGO pieces', image)

cv2.waitKey(0)
cv2.destroyAllWindows()


[ 15.  60. 147.]


In [105]:
import cv2
import os
import json
import numpy as np
def load_json(filename):
        with open(filename) as f:
            return json.load(f)
images_dir = './samples'
input = 'input.json'
canny_output = 'outputs/canny_output.json'
with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
for area_threshold in range(0, 500, 100):
    for kernel_size in range(3, 10, 2):
        for tolerance in range(0, 130, 5):
            results = []

            for image_index in range(len(image_paths)):
                contours_list = []
                original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))

                # Resize the image
                original_image = cv2.resize(original_image, (0, 0), fx=0.15, fy=0.15)
                image = original_image.copy()

                hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

                    # Flatten the HSV image to a 1D array
                hsv_flattened = hsv.reshape((-1, 3))

                # Mask out non-zero pixels (non-black pixels)
                hsv_flattened = hsv_flattened[hsv_flattened[:, 2] > 0]

                # Calculate the median color (background) in the image
                background_color = np.median(hsv_flattened, axis=0)
                lower_limit_0 = max(background_color[0] - tolerance,0)
                upper_limit_0 = min(background_color[0] + tolerance,255)
                lower_limit_1 = max(background_color[1] - tolerance,0)
                upper_limit_1 = min(background_color[1] + tolerance,255)
                lower_limit_2 = max(background_color[2] - tolerance,0)
                upper_limit_2 = min(background_color[2] + tolerance,255)

                lower_background = np.array([lower_limit_0, lower_limit_1, lower_limit_2])
                upper_background = np.array([upper_limit_0, upper_limit_1, upper_limit_2])


                blur = cv2.GaussianBlur(hsv, (kernel_size, kernel_size), 0)
                grey_mask = cv2.inRange(blur, lower_background, upper_background)

                non_grey_mask = cv2.bitwise_not(grey_mask)

                result = cv2.bitwise_and(image, image, mask=non_grey_mask)

                contours, _ = cv2.findContours(non_grey_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                for contour in contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    if(w*h < area_threshold):
                        continue
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    contours_list.append(contour)

                img_info = {
                        "file_name": image_paths[image_index],
                        "num_colors": len(contours_list),
                        "num_detections": len(contours_list),
                        "detected_objects": []  
                }
                results.append(img_info)    
            output = {
                "results": results
            }
            with open(f'outputs/canny_output_{tolerance}_{kernel_size}_{area_threshold}.json', 'w') as json_file:
                json.dump(output, json_file, indent=4)

            
            output_data = load_json(f'outputs/canny_output_{tolerance}_{kernel_size}_{area_threshold}.json')
            solutions_data = load_json('solutions.json')

            solutions_dict = {item['file_name']: item for item in solutions_data['results']}
            passed_tests = 0
            total_tests = len(output_data['results'])

            for file in output_data['results']:
                file_name = file['file_name']
                
                if file_name in solutions_dict:
                    same_colors = file['num_colors'] == solutions_dict[file_name]['num_colors']
                    same_detections = file['num_detections'] == solutions_dict[file_name]['num_detections']
                    if (same_detections):
                        passed_tests += 1
                    #  print(f"[PASS] {file_name}")
                    else:
                        pass
                    #  print(f"[FAIL] {file_name} - {file['num_detections']} detections, expected {solutions_dict[file_name]['num_detections']} detections")
                else:
                    total_tests -= 1
                    print(f"{file_name}: Not found in solutions")

            print(f"\n[SCORE] {passed_tests}/{total_tests} tests passed for tolerance {tolerance} and kernel size {kernel_size} and area threshold {area_threshold}\n")    



    




[SCORE] 6/50 tests passed for tolerance 0 and kernel size 3 and area threshold 0


[SCORE] 1/50 tests passed for tolerance 5 and kernel size 3 and area threshold 0


[SCORE] 0/50 tests passed for tolerance 10 and kernel size 3 and area threshold 0


[SCORE] 0/50 tests passed for tolerance 15 and kernel size 3 and area threshold 0


[SCORE] 1/50 tests passed for tolerance 20 and kernel size 3 and area threshold 0


[SCORE] 2/50 tests passed for tolerance 25 and kernel size 3 and area threshold 0


[SCORE] 2/50 tests passed for tolerance 30 and kernel size 3 and area threshold 0


[SCORE] 2/50 tests passed for tolerance 35 and kernel size 3 and area threshold 0


[SCORE] 2/50 tests passed for tolerance 40 and kernel size 3 and area threshold 0


[SCORE] 2/50 tests passed for tolerance 45 and kernel size 3 and area threshold 0


[SCORE] 5/50 tests passed for tolerance 50 and kernel size 3 and area threshold 0


[SCORE] 4/50 tests passed for tolerance 55 and kernel size 3 and area thresho

In [111]:
import cv2
import os
import json
import numpy as np

images_dir = './samples'
input = 'input.json'
canny_output = 'outputs/canny_output.json'
results = []
with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
for image_index in range(len(image_paths)):
    original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))

    # Resize the image
    original_image = cv2.resize(original_image, (0, 0), fx=0.15, fy=0.15)
    image = original_image.copy()

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply median blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    canny_image = image.copy()
    cannys_found_contours = [] 

    edges = cv2.Canny(blur, 30, 50)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        
        if area > 10:
            to_place = True
            cannys_found_contours_copy = cannys_found_contours.copy()
            for existing_contour_info in cannys_found_contours:
                existing_contour = existing_contour_info['contour']
                existing_x, existing_y, existing_w, existing_h = existing_contour_info['x'], existing_contour_info['y'], existing_contour_info['w'], existing_contour_info['h']
                
                intersect_x1 = max(x, existing_x)
                intersect_y1 = max(y, existing_y)
                intersect_x2 = min(x + w, existing_x + existing_w)
                intersect_y2 = min(y + h, existing_y + existing_h)
                intersect_area = max(0, intersect_x2 - intersect_x1) * max(0, intersect_y2 - intersect_y1)



                if intersect_area > 0:
                    if((w*h) > (existing_w * existing_h)):
                        cannys_found_contours_copy.remove(existing_contour_info)    
                    else:
                        to_place = False
                        break
                    

            if to_place:
                contour_info = {
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h,
                    'contour': contour 
                }

                cannys_found_contours_copy.append(contour_info)
                cannys_found_contours = cannys_found_contours_copy 

    img_info = {
        "file_name": image_paths[image_index],
        "num_colors": len(cannys_found_contours),
        "num_detections": len(cannys_found_contours),
        "detected_objects": []  
    }
    results.append(img_info)

output = {
    "results": results
}
with open(canny_output, 'w') as json_file:
    json.dump(output, json_file, indent=4)




